In [48]:
from src.nws import NWSClient, celsius_to_fahrenheit, fahrenheit_to_celsius
from src.params import *
from datetime import datetime, timedelta
import pytz

client = NWSClient()

In [49]:
# MIA - 11:07 AM CT
# MDW - 11:08 AM CT - still too uncertain
# AUS - 11:23 AM CT - still too uncertain
# NYC 

station = StationID.MDW
station_tz = STATION_TZ[station]
prior_days = 2

df = client.get_timeseries(station, datetime.now(tz=BASE_TZ) - timedelta(days=prior_days), datetime.now(tz=BASE_TZ))
df = df.rename(columns={
        'air_temp_set_1': 'temperature', 
        'relative_humidity_set_1': 'relative_humidity',
        'wind_speed_set_1': 'wind_speed',
        'wind_direction_set_1': 'wind_direction',
        'air_temp_high_6_hour_set_1': 'temperature_high_6_hour',
        'air_temp_high_24_hour_set_1': 'temperature_high_24_hour',

    })
df[['temperature', 'temperature_high_6_hour', 'temperature_high_24_hour']] = df[['temperature', 'temperature_high_6_hour', 'temperature_high_24_hour']].apply(celsius_to_fahrenheit)

df.info()
df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 629 entries, 2024-11-10 16:30:00-06:00 to 2024-11-12 16:20:00-06:00
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   temperature               629 non-null    float64
 1   relative_humidity         629 non-null    float64
 2   wind_speed                627 non-null    float64
 3   wind_direction            627 non-null    float64
 4   temperature_high_6_hour   8 non-null      float64
 5   temperature_high_24_hour  2 non-null      float64
 6   time                      629 non-null    object 
dtypes: float64(6), object(1)
memory usage: 39.3+ KB


,temperature,relative_humidity,wind_speed,wind_direction,temperature_high_6_hour,temperature_high_24_hour,time
date_time,,,,,,,
2024-11-10 16:30:00-06:00,55.4,58.29,7.202,260.0,NaN,NaN,16:30:00
2024-11-10 16:35:00-06:00,55.4,62.48,6.688,260.0,NaN,NaN,16:35:00
2024-11-10 16:40:00-06:00,55.4,62.48,8.746,270.0,NaN,NaN,16:40:00
2024-11-10 16:45:00-06:00,55.4,62.48,8.746,260.0,NaN,NaN,16:45:00
2024-11-10 16:50:00-06:00,55.4,62.48,7.717,270.0,NaN,NaN,16:50:00
...,...,...,...,...,...,...,...
2024-11-12 16:00:00-06:00,50.0,57.52,5.144,90.0,NaN,NaN,16:00:00
2024-11-12 16:05:00-06:00,50.0,57.52,3.601,100.0,NaN,NaN,16:05:00
2024-11-12 16:10:00-06:00,48.2,61.51,4.630,80.0,NaN,NaN,16:10:00


In [50]:
import pandas as pd
pd.options.plotting.backend = "plotly"

import plotly.express as px

fig = px.line(df, y="temperature", title=f"Temperature (°F) for Station {station}", width=1800, height=900)
fig.update_traces(mode="lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.update_yaxes(title="Temperature (°F)")
fig.update_xaxes(title="Date")

fig.show()


In [51]:
import plotly.express as px

max_idx = df.groupby(df.index.date)["temperature"].idxmax()

fig = px.line(df.loc[max_idx], y="temperature", title=f"Max Temperature (°F) Station {station}", width=1800, height=900)
fig.update_traces(mode="lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.update_yaxes(title="Temperature (°F)")
fig.update_xaxes(title="Date")
fig.show()



In [33]:
import pandas as pd
pd.options.plotting.backend = "plotly"

import plotly.express as px

df_6_hour = df[["temperature_high_6_hour"]].dropna()
fig = px.line(df_6_hour, y="temperature_high_6_hour", title="Temperature (°F)", width=1800, height=900)
fig.update_traces(mode="lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.update_yaxes(title="Temperature (°F)")
fig.update_xaxes(title="Date")

fig.show()


In [52]:
import pandas as pd
pd.options.plotting.backend = "plotly"

import plotly.express as px

df_24_hour = df[["temperature_high_24_hour"]].dropna()
fig = px.line(df_24_hour, y="temperature_high_24_hour", title="Temperature (°F)", width=1800, height=900)
fig.update_traces(mode="lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.update_yaxes(title="Temperature (°F)")
fig.update_xaxes(title="Date")

fig.show()
